In [4]:
#import required libraries
import numpy as np
import pandas as pd
import os
from PIL import Image
import cv2
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [5]:
#step_1:
#make a dataset of images in file directory


# Set the directory to save the dataset
dataset_dir = "C:/Users/asus/Desktop/ais-ipm"


# Define the colors and their corresponding labels
colors = {
    'yellow': 0,
    'red': 1,
    'blue': 2,
    'green': 3,
    'black': 4,
    'white': 5
}

# Generate and save the images
for color in colors:
    label = colors[color]
    for i in range(100):  # Generate 100 images per color
        image = Image.new('RGB', (64, 64), color)
        filename = f'{color}_{i}.jpg'
        filepath = os.path.join(dataset_dir, filename)
        image.save(filepath)

        print(f'Saved {filepath} with label {label}')


Saved C:/Users/asus/Desktop/ais-ipm\yellow_0.jpg with label 0
Saved C:/Users/asus/Desktop/ais-ipm\yellow_1.jpg with label 0
Saved C:/Users/asus/Desktop/ais-ipm\yellow_2.jpg with label 0
Saved C:/Users/asus/Desktop/ais-ipm\yellow_3.jpg with label 0
Saved C:/Users/asus/Desktop/ais-ipm\yellow_4.jpg with label 0
Saved C:/Users/asus/Desktop/ais-ipm\yellow_5.jpg with label 0
Saved C:/Users/asus/Desktop/ais-ipm\yellow_6.jpg with label 0
Saved C:/Users/asus/Desktop/ais-ipm\yellow_7.jpg with label 0
Saved C:/Users/asus/Desktop/ais-ipm\yellow_8.jpg with label 0
Saved C:/Users/asus/Desktop/ais-ipm\yellow_9.jpg with label 0
Saved C:/Users/asus/Desktop/ais-ipm\yellow_10.jpg with label 0
Saved C:/Users/asus/Desktop/ais-ipm\yellow_11.jpg with label 0
Saved C:/Users/asus/Desktop/ais-ipm\yellow_12.jpg with label 0
Saved C:/Users/asus/Desktop/ais-ipm\yellow_13.jpg with label 0
Saved C:/Users/asus/Desktop/ais-ipm\yellow_14.jpg with label 0
Saved C:/Users/asus/Desktop/ais-ipm\yellow_15.jpg with label 0
Sa

In [6]:
#step_2:
#make dataset of images and their labels

# Set the directory where the dataset is located
dataset_dir = "C:/Users/asus/Desktop/ais-ipm"

# Load images and labels into lists
images = []
labels = []
colors = ['yellow', 'red', 'blue', 'green', 'black', 'white']
for color_id, color in enumerate(colors):
    for i in range(100):
        filename = f'{color}_{i}.jpg'
        filepath = os.path.join(dataset_dir, filename)
        image = Image.open(filepath)
        images.append(np.array(image))
        labels.append(color_id)

# Convert lists to NumPy arrays
images = np.array(images)
labels = np.array(labels)


In [7]:
#step_3:
#preprocessing the input data to neural network 

# Split the dataset into training and test sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.25, random_state=42)

# Normalize pixel values between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels to one-hot encoding
num_classes = len(colors)
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


In [8]:
#step_4:
#defining neural network

# Create a convolutional neural network model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [9]:
#step_5:
#fit the model to data

# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

# predicted = model.predict(x_test)
# test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
# print("Test loss:", test_loss)
# print("Test accuracy:", test_acc)

Epoch 1/10
15/15 [==============================] - 2s 85ms/step - loss: 0.7141 - accuracy: 0.7444 - val_loss: 0.2409 - val_accuracy: 1.0000
Epoch 2/10
15/15 [==============================] - 1s 74ms/step - loss: 0.2463 - accuracy: 1.0000 - val_loss: 0.0807 - val_accuracy: 1.0000
Epoch 3/10
15/15 [==============================] - 1s 75ms/step - loss: 0.0276 - accuracy: 1.0000 - val_loss: 3.5566e-04 - val_accuracy: 1.0000
Epoch 4/10
15/15 [==============================] - 1s 81ms/step - loss: 1.0560e-04 - accuracy: 1.0000 - val_loss: 1.8762e-05 - val_accuracy: 1.0000
Epoch 5/10
15/15 [==============================] - 1s 77ms/step - loss: 1.4798e-05 - accuracy: 1.0000 - val_loss: 1.0079e-05 - val_accuracy: 1.0000
Epoch 6/10
15/15 [==============================] - 1s 84ms/step - loss: 9.0598e-06 - accuracy: 1.0000 - val_loss: 7.8129e-06 - val_accuracy: 1.0000
Epoch 7/10
15/15 [==============================] - 1s 81ms/step - loss: 7.3247e-06 - accuracy: 1.0000 - val_loss: 6.5294e-06 

In [10]:
#step_6:
#add noise and evaluate the performance:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    brightness_range=(0.8, 1.2),
    channel_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True
)

datagen.fit(x_train)
model.fit(datagen.flow(x_train, y_train, batch_size=32), epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
15/15 [==============================] - 2s 125ms/step - loss: 3.1987 - accuracy: 0.2844 - val_loss: 0.4292 - val_accuracy: 0.8600
Epoch 2/10
15/15 [==============================] - 1s 92ms/step - loss: 1.6024 - accuracy: 0.2756 - val_loss: 0.2897 - val_accuracy: 1.0000
Epoch 3/10
15/15 [==============================] - 1s 100ms/step - loss: 1.5496 - accuracy: 0.3044 - val_loss: 0.3990 - val_accuracy: 1.0000
Epoch 4/10
15/15 [==============================] - 2s 98ms/step - loss: 1.5993 - accuracy: 0.2756 - val_loss: 0.4385 - val_accuracy: 0.8133
Epoch 5/10
15/15 [==============================] - 2s 105ms/step - loss: 1.5355 - accuracy: 0.3067 - val_loss: 0.4262 - val_accuracy: 0.8133
Epoch 6/10
15/15 [==============================] - 1s 97ms/step - loss: 1.5506 - accuracy: 0.2978 - val_loss: 0.4201 - val_accuracy: 0.8133
Epoch 7/10
15/15 [==============================] - 2s 110ms/step - loss: 1.5544 - accuracy: 0.2956 - val_loss: 0.4089 - val_accuracy: 0.8133
Epoch 8/1